In [116]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions

client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [117]:
program_id = PublicKey("FV55U1fovcccV4Nzcg1EqJ8qGnGSCAP56jemkp2EFoLK")
program_id_account_meta = AccountMeta(program_id, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
program_id_account_meta

AccountMeta(pubkey=FV55U1fovcccV4Nzcg1EqJ8qGnGSCAP56jemkp2EFoLK, is_signer=False, is_writable=False)

In [118]:
spl_program_meta

AccountMeta(pubkey=TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA, is_signer=False, is_writable=False)

In [119]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 600000000)

mint_account = Keypair.generate()

seed1 = bytes([225,215,211,54,213,18,25,122]*4)
vault_keypair = Keypair.from_seed(seed1)

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
vault_account_meta = AccountMeta(Keypair.generate().public_key, False, True)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)

# transaction = Transaction()
# transaction.add(
#   create_account(CreateAccountParams(from_pubkey=payer_account_meta.pubkey, new_account_pubkey=mint_account.public_key, lamports=Token.get_min_balance_rent_for_exempt_for_mint(client), space=MINT_LAYOUT.sizeof(), program_id=spl_program_meta.pubkey))
# )
# transaction.add(
#   initialize_mint(InitializeMintParams(
#     decimals=6, 
#     program_id=spl_program_meta.pubkey, 
#     mint=mint_account.public_key,
#     mint_authority=program_id, 
#     freeze_authority=program_id
#   ))
# )
   
# client.send_transaction(transaction, payer_keypair, mint_account)

In [120]:
mint_account_meta = AccountMeta(mint_account.public_key, True, True)
associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, mint_account.public_key)
associated_account_meta = AccountMeta(associated_token_address, False, True)

(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)


accounts = [
    # program_id_account_meta,
    payer_account_meta, 
    vault_account_meta, 
    mint_account_meta, 
    rent_account_meta, 
    associated_account_meta,
    spl_program_meta,
    AccountMeta(SYS_PROGRAM_ID, False, False),
    AccountMeta(mint_authority, False, False),

    spl_program_meta,
    AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
    spl_program_meta,
    ]
    
transaction = Transaction()
transaction.add(TransactionInstruction(
    accounts,
    program_id,
    bytes([])
))
client.send_transaction(transaction, payer_keypair, mint_account)

{'jsonrpc': '2.0',
 'result': 'eTSFSS5vPUdynWSyPjgengACbaoXf8jeNFVV4dypsyHwNTepLbWNE7m1aYVpUTMFFfo1sEN4povyJ3AUYjtZJTF',
 'id': 3}

In [121]:
(mint_authority, nonce) = PublicKey.find_program_address([b'cyrial_pda'], program_id)
mint_authority, nonce

(7Hrb8JN8xHKyfccJBgRiwKcmYsWov9U9Fmsr1xx4TBB4, 254)

In [122]:
payer_keypair.public_key

2ASw3tjK5bSxQxFEMsM6J3DnBozNh7drVErSwc7AtzJv